In [ ]:
import pandas as pd
import numpy as np

## Data Load

In [ ]:
!unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip

In [ ]:
PATH = '/kaggle/working/'

In [ ]:
train = pd.read_csv(PATH+'labeledTrainData.tsv', delimiter='\t', quoting=3)
test = pd.read_csv(PATH+'testData.tsv', delimiter='\t', quoting=3)

In [ ]:
train.head(3)

In [ ]:
train.info()

In [ ]:
train.describe(exclude=[np.number])

In [ ]:
train['sentiment'].value_counts()

In [ ]:
train['review'][0][:700]

## 전처리 sample
* BeautifulSoup 으로 html 태그 제거
* 정규표현식으로 알파벳 이외의 문자 공백 제거
* NLTK 데이터를 활용 불용어(stopwords) 제거
* Stemming 을 활용하여 어간 추출

In [ ]:
!pip show BeautifulSoup4

In [ ]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train['review'][0], 'html5lib')
print(train['review'][0][:700])
example1.get_text()[:700]

In [ ]:
import re

letters_only = re.sub('[^a-zA-Z]', ' ', example1.get_text())
letters_only[:700]

In [ ]:
lower_case = letters_only.lower()
words = lower_case.split()
print(len(words))
words[:10]

In [ ]:
!pip show nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')[:10]

In [ ]:
words = [w for w in words if not w in stopwords.words('english')]
print(len(words))
words[:10]

## Stemming

In [ ]:
# 퍼터 스태머
stemmer = nltk.stem.PorterStemmer()
print(stemmer.stem('maximum'))
print('The stemmed form of running is : {}'.format(stemmer.stem('running')))
print('The stemmed form of runs is : {}'.format(stemmer.stem('runs')))
print('The stemmed form of run is : {}'.format(stemmer.stem('run')))

In [ ]:
# 랭커스터 스태머
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
print(lancaster_stemmer.stem('maximum'))
print('The stemmmed form of running is : {}'.format(lancaster_stemmer.stem('running')))
print('The stemmed form of runs is : {}'.format(lancaster_stemmer.stem('runs')))
print('The stemmed form of run is {}'.format(lancaster_stemmer.stem('run')))

In [ ]:
words[:10]

In [ ]:
# snowball stemmer
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
words = [stemmer.stem(w) for w in words]

# snowball stemm 처리 후
words[:10]

## Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

print(wordnet_lemmatizer.lemmatize('fly'))
print(wordnet_lemmatizer.lemmatize('flies'))

words = [wordnet_lemmatizer.lemmatize(w) for w in words]
words[:10]

## 전처리 All
* BeautifulSoup 으로 html 태그 제거
* 정규표현식으로 알파벳 이외의 문자 공백 제거
* 소문자 변환
* NLTK 데이터를 활용 불용어(stopwords) 제거
* Stemming 을 활용하여 어간 추출

In [ ]:
def review_to_words(raw_review):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자가 아닌 문자는 공백으로 변환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. 소문자 변환
    words = letters_only.lower().split()
    # 4. stopwords 를 세트로 변환
    stops = set(stopwords.words('english'))
    # 5. stopwords 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 어간 추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return (' '.join(stemming_words))

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
clean_review = review_to_words(train['review'][0])
clean_review

In [ ]:
# 첫번째 리뷰를 대상으로 전처리 해줬던 내용을 전체 텍스트 데이터를 대상으로 처리한다.
# 전체 리뷰 데이터 수 가져오기
num_reviews = train['review'].size
num_reviews

In [ ]:
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    # 키워드 항목 중 workers 파라미터를 꺼냄
    workers = kwargs.pop('workers')
    # 위에서 가져온 workers 수로 프로세스 풀을 정의
    pool = Pool(processes=workers)
    # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
    result = pool.map(_apply_df, [(d, func, kwargs)
                                 for d in np.array_split(df, workers)])
    pool.close()
    # 작업 결과를 합쳐서 반환
    return pd.concat(list(result))

In [ ]:
%time
clean_train_reviews = apply_by_multiprocessing(\
    train['review'], review_to_words, workers=4)

In [ ]:
%time
clean_test_reviews = apply_by_multiprocessing(\
    test['review'], review_to_words, workers=4)

## WordCloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

def displayWordCloud(data=None, backgroundcolor='white', width=800, height=600):
    wordcloud = WordCloud(stopwords=STOPWORDS,
                         background_color=backgroundcolor,
                         width=width, height=height).generate(data)
    plt.figure(figsize=(15,10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
# 학습 데이터의 모든 단어에 대한 워드 클라우드
%time
displayWordCloud(' '.join(clean_train_reviews))

In [ ]:
# 단어 수
train['num_words'] = clean_train_reviews.apply(lambda x:len(str(x).split()))
#  중복을 제거한 단어수
train['num_uniq_words'] = clean_train_reviews.apply(lambda x:len(set(str(x).split())))

In [ ]:
# 테스트 데이터의 모든 단어에 대한 워드 클라우드
%time
displayWordCloud(' '.join(clean_test_reviews))

In [ ]:
x = clean_train_reviews[0]
x = str(x).split()
print(len(x))
x[:10]

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(18,6)
print('리뷰별 단어 평균 값 : ', train['num_words'].mean())
print('리뷰별 단어 중간 값 : ', train['num_words'].median())
sns.distplot(train['num_words'], bins=100, ax=axes[0])
axes[0].axvline(train['num_words'].median(), linestyle='dashed')
axes[0].set_title('review unique word distribution')


print('review unique word mean:', train['num_uniq_words'].mean())
print('review unique word meidan', train['num_uniq_words'].median())
sns.distplot(train['num_uniq_words'], bins=100, color='g', ax=axes[1])
axes[1].axvline(train['num_uniq_words'].median(), linestyle='dashed')
axes[1].set_title('review unique word distribution')

## Bag-of-words model
### 사이킷런의 CountVenctorizer 를 통해 피처 생성
* 정규표현식을 사용해 토큰을 추출
* 모두 소문자로 변환시키기 때문에 good, Good, gOod이 모두 같은 특성이 됨
* 의미없는 특성을 많이 생성하기 때문에 적어도 두 개의 문서에 나타난 토큰만을 사용
* min_df로 토큰이 나타날 최소 문서 개수를 지정할 수 있음

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            min_df = 2,
                            ngram_range=(1,3),
                            max_features = 20000)
vectorizer

In [ ]:
pipeline = Pipeline([
    ('vect', vectorizer),
])

In [ ]:
%time
train_data_features = pipeline.fit_transform(clean_train_reviews)
train_data_features

In [ ]:
train_data_features.shape

In [ ]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

In [ ]:
# 벡터화된 피처를 확인
dist = np.sum(train_data_features, axis=0)

for tag, count in zip(vocab, dist):
    print(count, tag)

pd.DataFrame(dist, columns=vocab)

In [ ]:
pd.DataFrame(train_data_features[:10].toarray(), columns=vocab).head()

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(
    n_estimators=100, n_jobs=-1, random_state=2018)
forest

In [ ]:
%time
forest = forest.fit(train_data_features, train['sentiment'])

In [ ]:
from sklearn.model_selection import cross_val_score
%time
score = np.mean(cross_val_score(\
    forest, train_data_features, \
    train['sentiment'], cv=10, scoring='roc_auc'))
score

## Prediction

In [ ]:
clean_test_reviews[0]

In [ ]:
%time
test_data_features = pipeline.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [ ]:
test_data_features

In [ ]:
# 벡터화 된 단어로 숫자가 문서에서 등장하는 횟수를 나타낸다
test_data_features[5][:100]

In [ ]:
# 벡터화 하며 만든 사전에서 해당 단어가 무엇인지 찾아볼 수 있다.
vocab[8], vocab[2558], vocab[2559], vocab[2560]

In [ ]:
# 테스트 데이터를 넣고 예측한다.
result = forest.predict(test_data_features)
result[:10]

## 캐글 제출을 위해 예측결과 저장

In [ ]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

In [ ]:
output.to_csv('tutorial1_BOW_{0:.5f}.csv'.format(score), index=False, quoting=3)

In [ ]:
output_sentiment = output['sentiment'].value_counts()
print(np.abs(output_sentiment[0] - output_sentiment[1]))
output_sentiment

## Train, Test 의 감정분류 결과 값 비교

In [ ]:
fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.countplot(train['sentiment'], ax=axes[0])
sns.countplot(output['sentiment'], ax=axes[1])